In [27]:
import pandas as pd
import numpy as np
import tarfile
import requests
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import itertools
from pprint import pprint

# Todo
- [x] create a ColumnSelector class
- [x] use simple imputer to fill in na values
- [x] Normalise all numeric columns
- [ ] Build pipeline
- [ ] Run KNN Regressor

In [3]:
HOUSING_URL = "https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.tgz"
DATA_PATH = Path("./datasets/housing")
def fetch_housing_data(data_url=HOUSING_URL, file_path=DATA_PATH):
    data = requests.get(data_url).content
    file_path.mkdir(exist_ok=True, parents=True)
    tgz_path = f"{DATA_PATH}/housing.tgz"
    with open(tgz_path, "wb") as tgz_file:
        tgz_file.write(data)    
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=file_path)
    housing_tgz.close()

fetch_housing_data()

In [4]:
def load_housing_data(housing_path=DATA_PATH):
    return pd.read_csv(f"{housing_path}/housing.csv")

housing = load_housing_data()

In [5]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [6]:
X = housing.drop(["median_house_value"], axis=1)
y = housing["median_house_value"]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
print(
f"""
    X size: {X.size}
    x train: {x_train.size / X.size} ({x_train.size})
    x test: {x_test.size / X.size} ({x_test.size})
"""
)


    X size: 185760
    x train: 0.8 (148608)
    x test: 0.2 (37152)



In [8]:
x_train.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 14196 to 15795
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16512 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
 8   ocean_proximity     16512 non-null  object 
dtypes: float64(8), object(1)
memory usage: 1.3+ MB


In [9]:
numeric_cols = list(x_train.drop(["ocean_proximity"], axis=1).columns)

In [10]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, desired_columns):
        self.desired_columns = desired_columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.desired_columns].values


In [11]:
pipeline = Pipeline([
    ("column_selector", ColumnSelector(numeric_cols)),
    ("imputer", SimpleImputer(strategy="median")),
    ("standard scalers", StandardScaler())
])
x_train_prepared = pipeline.fit_transform(x_train)
x_test_prepared = pipeline.fit_transform(x_test)

In [12]:
def test_knn(desired_columns, weights="uniform", n_neighbors=3):
    pipeline = Pipeline([
        ("column_selector", ColumnSelector(numeric_cols)),
        ("imputer", SimpleImputer(strategy="median")),
        ("standard scalers", StandardScaler())
    ])
    x_train_prepared = pipeline.fit_transform(x_train)
    x_test_prepared = pipeline.fit_transform(x_test)
    
    knn_regressor = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights)
    knn_regressor.fit(x_train_prepared, y_train)
    housing_predictions = knn_regressor.predict(x_test_prepared)
    mse = mean_squared_error(y_test, housing_predictions)
    rmse = np.sqrt(mse)
    return rmse
    

In [13]:
knn_regressor = KNeighborsRegressor(n_neighbors=3)
knn_regressor.fit(x_train_prepared, y_train)

KNeighborsRegressor(n_neighbors=3)

In [19]:
housing_predictions = knn_regressor.predict(x_test_prepared)
mse = mean_squared_error(y_test, housing_predictions)
rmse = np.sqrt(mse)
rmse

64725.78694113039

In [20]:
print(numeric_cols)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']


In [21]:
combinations = [
    numeric_cols,
    ["housing_median_age", "total_rooms", "total_bedrooms", "median_income"],
    ["housing_median_age", "total_rooms", "population"],
]

In [26]:
results = []
for combination in combinations:
    for weighting_approach in ["uniform", "distance"]:
        rmse = test_knn(combination, weights=weighting_approach)
        results.append((combination, weighting_approach, rmse))

In [30]:
pprint(results)

[(['longitude',
   'latitude',
   'housing_median_age',
   'total_rooms',
   'total_bedrooms',
   'population',
   'households',
   'median_income'],
  'uniform',
  64725.78694113039),
 (['longitude',
   'latitude',
   'housing_median_age',
   'total_rooms',
   'total_bedrooms',
   'population',
   'households',
   'median_income'],
  'distance',
  64460.36447674085),
 (['housing_median_age', 'total_rooms', 'total_bedrooms', 'median_income'],
  'uniform',
  64725.78694113039),
 (['housing_median_age', 'total_rooms', 'total_bedrooms', 'median_income'],
  'distance',
  64460.36447674085),
 (['housing_median_age', 'total_rooms', 'population'],
  'uniform',
  64725.78694113039),
 (['housing_median_age', 'total_rooms', 'population'],
  'distance',
  64460.36447674085)]
